In [233]:
import pandas as pd
import numpy as np

## Основные константы блока
<ul>
<li><b>strWebSrc</b> - адрес исходника в WWW</li>
<li><b>strNetSource</b> - адрес локального исходника</li>
<li><b>iErrorToInt</b> - число-заполнитель отсутствующих кодов групп и показателей (для удобства использования в индексации)</li>
<li><b>dctIndexColumnsNames</b> - словарь универсальных имен колонок для мультииндекса, в эти колонки разбивается код ОКВЭД</li>
<li><b>dctResultFields</b> - словарь универсальных имен колонок рабочих, часто используемых столбцов набора данных </li>
</ul>

In [429]:
strWebSrc=r'http://data.gov.ru/sites/default/files/opendata/7710168515-Obscherossiyskiyklassifikatorvidovehkonomicheskoydeyatelnosti2014/data-2016-10-17T00-00-00-structure-2016-10-17T00-00-00.json'
#strNetSource=r'smb://commd/statistica/99-Information/ОКВЭД2/ОКВЭД2.xls'
strNetSource=r'/home/egor/csv_source/ОКВЭД2.xls'
iErrorToInt=-1

dctIndexColumnsNames={'CATEGORY':'cat', 'GROUP1':'grp1', 'GROUP2':'grp2', 'CODE':'num'}
dctResultFields={'okvedF':'Idx', 'okved':'Code', 'name':'Name'}

## Читаем коды ОКВЭД в pandas DataFrame
две функции для двух разных источников: <BR/>
1 - с WWW в формате json (как на сайте http://data.gov.ru/) <BR/>
2 - из локального файла Excel (форматы группы Сальникова) <BR/>

## Основное в формате готового DataFrame:
Код ОКВЭД разбивается (разделитель - точка) на четыре группы: <BR/>
1 - категория (латинская заглавная буква)<BR/>
2 - код группы 1 <BR/>
3 - код группы 2 <BR/>
3 - код поакзателя <BR/>
<b>Коды групп и показателей преобразуются в тип INT, отсутствующий код замещяется iErrorToInt</b>

In [520]:
def to_int(x):
    try:
        return int(x)
    except:
        return iErrorToInt
        
def read_from_web(strSource):
    """читает данные в формате json, готовит pandas DataFrame
    разбивает поле Idx (код ОКВЭД) на 4 поля: категория, группа 1, группа 2, код показателя
    коды групп и показателя - число;
    четыре поля кода ОКВЭД => мультииндекс"""
        
    df=pd.read_json(strSource, orient='records', encoding='Cp1251')
    df=df.drop(['system_object_id', 'signature_date', 'global_id', 'Razdel'], axis=1) # возможно, это нужные поля
    df.rename(columns={'Kod':dctResultFields['okved']}, inplace=True)
    df[dctIndexColumnsNames['CATEGORY']], df[dctIndexColumnsNames['GROUP1']], \
    df[dctIndexColumnsNames['GROUP2']], df[dctIndexColumnsNames['CODE']]=df['Idx'].str.split('.').str
    
    df[ list(dctIndexColumnsNames.values())[1:] ]=df[list(dctIndexColumnsNames.values())[1:]].applymap(to_int)
    df[dctIndexColumnsNames['CATEGORY']].astype('category')
    df.set_index( list(dctIndexColumnsNames.values()), inplace=True)
    return df.sort_index()

#print(read_from_web(strWebSrc))

def read_from_excel(strSource):
    """читает данные в формате xls (НЕ XLSB!!!) из файла В. Сальникова - Дю Галимова, 
    готовит pandas DataFrame
    формирует поле Idx (код ОКВЭД) на основе поля КОД файла, 
    формирует поля: категория, группа 1, группа 2, код показателя
    коды групп и показателя - число;
    четыре поля кода ОКВЭД => мультииндекс
    осталось вычистить от повторов - !!!
    """
    def make_cat_field(df1):
        dfQ=df1[df1[dctIndexColumnsNames['CATEGORY']].notnull()]
        lst=dfQ.index.tolist()+[len(df1)]
        i=0
        while i<len(lst)-1:
            df1.loc[lst[i]:lst[i+1]-1,dctIndexColumnsNames['CATEGORY']]=dfQ.loc[lst[i], dctIndexColumnsNames['CATEGORY']]
            i+=1
        df1.loc[lst[-1]-1,dctIndexColumnsNames['CATEGORY']]=dfQ.loc[lst[i-1], dctIndexColumnsNames['CATEGORY']]
        df1['cat'].fillna('0', inplace=True)

        
    df=pd.read_excel(strSource, skiprows=[0,], header=0)
    df.rename(columns={'Код':dctResultFields['okved'], 'Название группировки':dctResultFields['name']}, inplace=True)
    strEx=r'(?P<{CATEGORY}>[A-Z])|(?P<{GROUP1}>\d+)\.?(?P<{GROUP2}>\d+)?\.?(?P<{CODE}>\d+)?'.format(**dctIndexColumnsNames)
    df[list(dctIndexColumnsNames.values())] = df[dctResultFields['okved']].str.extract(strEx, expand=True)
    make_cat_field(df)
    df[dctResultFields['okvedF']]=df[list(dctIndexColumnsNames.values())].fillna('').apply(lambda x: '.'.join(x), axis=1)
    df[dctResultFields['okvedF']]=df[dctResultFields['okvedF']].str.replace(r'\.+$', '')
    
    df[ list(dctIndexColumnsNames.values())[1:] ]=df[list(dctIndexColumnsNames.values())[1:]].applymap(to_int)
    df[dctIndexColumnsNames['CATEGORY']].astype('category')
    df.set_index( list(dctIndexColumnsNames.values()), inplace=True)
    return df
    
print (read_from_excel(strNetSource))




                                                                Name  \
cat grp1 grp2 num                                                      
0    0   -1   -1                                               ВСЕГО   
A   -1   -1   -1   СЕЛЬСКОЕ, ЛЕСНОЕ ХОЗЯЙСТВО, ОХОТА, РЫБОЛОВСТВО...   
     1   -1   -1   Растениеводство и животноводство, охота и пред...   
          1   -1                      Выращивание однолетних культур   
          11  -1   Выращивание зерновых (кроме риса), зернобобовы...   
               1                        Выращивание зерновых культур   
               11                                Выращивание пшеницы   
               12                                 Выращивание ячменя   
               13                                    Выращивание ржи   
               14                               Выращивание кукурузы   
               15                                   Выращивание овса   
               16                                Выращивание гре

## Примеры использования полученного DataFrame

In [217]:
res_df=read_from_web(strWebSrc)
x=pd.IndexSlice

#выборка только категорий
#print(res_df.loc[x[:,iErrorToInt,iErrorToInt,iErrorToInt], x[:]])
print(res_df.loc[ 'A', 1, 11] )

            Idx       Kod                                               Name  \
num                                                                            
-1      A.01.11     01.11  Выращивание зерновых (кроме риса), зернобобовы...   
 1    A.01.11.1   01.11.1                       Выращивание зерновых культур   
 2    A.01.11.2   01.11.2                   Выращивание зернобобовых культур   
 3    A.01.11.3   01.11.3                Выращивание семян масличных культур   
 11  A.01.11.11  01.11.11                                Выращивание пшеницы   
 12  A.01.11.12  01.11.12                                 Выращивание ячменя   
 13  A.01.11.13  01.11.13                                    Выращивание ржи   
 14  A.01.11.14  01.11.14                               Выращивание кукурузы   
 15  A.01.11.15  01.11.15                                   Выращивание овса   
 16  A.01.11.16  01.11.16                                Выращивание гречихи   
 19  A.01.11.19  01.11.19               